In [2]:
%tensorflow_version 2.x  # this line is not required unless you are in a notebook
from keras.preprocessing import sequence
import keras
import tensorflow as tf
import os
import numpy as np

`%tensorflow_version` only switches the major version: 1.x or 2.x.
You set: `2.x  # this line is not required unless you are in a notebook`. This will be interpreted as: `2.x`.


TensorFlow 2.x selected.


In [3]:
path_to_file = tf.keras.utils.get_file('shakespeare.txt', 'https://storage.googleapis.com/download.tensorflow.org/data/shakespeare.txt')

1122304/1115394 [==============================] - 0s 0us/step


In [ ]:
# OR we can choose a file from local computer
from google.colab import files
path_to_file = list(files.upload().keys())[0]

In [5]:
# Read, then decode for py2 compat.
text = open(path_to_file, 'rb').read().decode(encoding='utf-8')
# length of text is the number of characters in it
print ('Length of text: {} characters'.format(len(text)))

Length of text: 1115394 characters


In [6]:
# Take a look at the first 250 characters in text
print(text[:250])

First Citizen:
Before we proceed any further, hear me speak.

All:
Speak, speak.

First Citizen:
You are all resolved rather to die than to famish?

All:
Resolved. resolved.

First Citizen:
First, you know Caius Marcius is chief enemy to the people.



In [7]:
vocab = sorted(set(text))
# Creating a mapping from unique characters to indices
char2idx = {u:i for i, u in enumerate(vocab)}
idx2char = np.array(vocab)

def text_to_int(text):
  return np.array([char2idx[c] for c in text])

text_as_int = text_to_int(text)

In [8]:
# lets look at how part of our text is encoded
print("Text:", text[:13])
print("Encoded:", text_to_int(text[:13]))

Text: First Citizen
Encoded: [18 47 56 57 58  1 15 47 58 47 64 43 52]


In [9]:
def int_to_text(ints):
  try:
    ints = ints.numpy()
  except:
    pass
  return ''.join(idx2char[ints])

print(int_to_text(text_as_int[:13]))

First Citizen


In [10]:
seq_length = 100  # length of sequence for a training example
examples_per_epoch = len(text)//(seq_length+1)

# Create training examples / targets
char_dataset = tf.data.Dataset.from_tensor_slices(text_as_int)

In [11]:
sequences = char_dataset.batch(seq_length+1, drop_remainder=True)

In [12]:
def split_input_target(chunk):  # for the example: hello
    input_text = chunk[:-1]  # hell
    target_text = chunk[1:]  # ello
    return input_text, target_text  # hell, ello

dataset = sequences.map(split_input_target)  # we use map to apply the above function to every entry

In [13]:
for x, y in dataset.take(2):
  print("\n\nEXAMPLE\n")
  print("INPUT")
  print(int_to_text(x))
  print("\nOUTPUT")
  print(int_to_text(y))



EXAMPLE

INPUT
First Citizen:
Before we proceed any further, hear me speak.

All:
Speak, speak.

First Citizen:
You

OUTPUT
irst Citizen:
Before we proceed any further, hear me speak.

All:
Speak, speak.

First Citizen:
You 


EXAMPLE

INPUT
are all resolved rather to die than to famish?

All:
Resolved. resolved.

First Citizen:
First, you 

OUTPUT
re all resolved rather to die than to famish?

All:
Resolved. resolved.

First Citizen:
First, you k


In [14]:
BATCH_SIZE = 64
VOCAB_SIZE = len(vocab)  # vocab is number of unique characters
EMBEDDING_DIM = 256
RNN_UNITS = 1024

# Buffer size to shuffle the dataset
# (TF data is designed to work with possibly infinite sequences,
# so it doesn't attempt to shuffle the entire sequence in memory. Instead,
# it maintains a buffer in which it shuffles elements).
BUFFER_SIZE = 10000

data = dataset.shuffle(BUFFER_SIZE).batch(BATCH_SIZE, drop_remainder=True)

In [15]:
def build_model(vocab_size, embedding_dim, rnn_units, batch_size):
  model = tf.keras.Sequential([
    tf.keras.layers.Embedding(vocab_size, embedding_dim,
                              batch_input_shape=[batch_size, None]),
    tf.keras.layers.LSTM(rnn_units,
                        return_sequences=True,
                        stateful=True,
                        recurrent_initializer='glorot_uniform'),
    tf.keras.layers.Dense(vocab_size)
  ])
  return model

model = build_model(VOCAB_SIZE,EMBEDDING_DIM, RNN_UNITS, BATCH_SIZE)
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (64, None, 256)           16640     
_________________________________________________________________
lstm (LSTM)                  (64, None, 1024)          5246976   
_________________________________________________________________
dense (Dense)                (64, None, 65)            66625     
Total params: 5,330,241
Trainable params: 5,330,241
Non-trainable params: 0
_________________________________________________________________


In [16]:
for input_example_batch, target_example_batch in data.take(1):
  example_batch_predictions = model(input_example_batch)  # ask our model for a prediction on our first batch of training data (64 entries)
  print(example_batch_predictions.shape, "# (batch_size, sequence_length, vocab_size)")  # print out the output shape

(64, 100, 65) # (batch_size, sequence_length, vocab_size)


In [17]:
# we can see that the predicition is an array of 64 arrays, one for each entry in the batch
print(len(example_batch_predictions))
print(example_batch_predictions)

64
tf.Tensor(
[[[ 2.45939032e-03 -8.70835967e-04 -4.71046474e-03 ...  4.52589896e-03
    1.36262085e-03  6.56182994e-04]
  [ 6.59943232e-03  2.97048478e-04  5.33989351e-03 ...  4.69408510e-03
   -1.91215798e-03  5.00021596e-03]
  [ 9.05392505e-03 -8.35602579e-04  3.35281785e-03 ...  2.43034307e-03
   -1.23294890e-02  5.74197993e-03]
  ...
  [ 2.16487306e-03 -1.56734977e-03  5.15085692e-03 ...  6.05046470e-03
   -1.58647802e-02  6.52140146e-03]
  [-1.30376057e-03 -2.81111314e-03  1.61838578e-03 ...  7.56839337e-03
   -5.50476881e-03  8.02735146e-03]
  [-2.84257485e-03 -5.41110151e-03  6.80038705e-04 ...  1.41270962e-02
   -2.90841889e-03  1.26127638e-02]]

 [[-2.25370808e-04 -3.71366041e-03 -9.37821242e-05 ...  8.94474145e-03
    2.08685687e-03  5.44433109e-03]
  [-5.78725431e-03 -6.64477237e-03  3.34943319e-03 ...  1.41648855e-02
    1.71536882e-03  1.68450829e-03]
  [-1.39602029e-03 -5.10429312e-03  5.78102563e-03 ...  1.25579070e-02
   -6.56377850e-03  5.68584353e-03]
  ...
  [ 4.069

In [18]:
# lets examine one prediction
pred = example_batch_predictions[0]
print(len(pred))
print(pred)
# notice this is a 2d array of length 100, where each interior array is the prediction for the next character at each time step

100
tf.Tensor(
[[ 0.00245939 -0.00087084 -0.00471046 ...  0.0045259   0.00136262
   0.00065618]
 [ 0.00659943  0.00029705  0.00533989 ...  0.00469409 -0.00191216
   0.00500022]
 [ 0.00905393 -0.0008356   0.00335282 ...  0.00243034 -0.01232949
   0.00574198]
 ...
 [ 0.00216487 -0.00156735  0.00515086 ...  0.00605046 -0.01586478
   0.0065214 ]
 [-0.00130376 -0.00281111  0.00161839 ...  0.00756839 -0.00550477
   0.00802735]
 [-0.00284257 -0.0054111   0.00068004 ...  0.0141271  -0.00290842
   0.01261276]], shape=(100, 65), dtype=float32)


In [19]:
# and finally well look at a prediction at the first timestep
time_pred = pred[0]
print(len(time_pred))
print(time_pred)
# and of course its 65 values representing the probabillity of each character occuring next

65
tf.Tensor(
[ 2.4593903e-03 -8.7083597e-04 -4.7104647e-03  2.6707388e-03
  3.2696538e-03 -2.0349703e-03 -3.3969278e-03  2.2507881e-04
 -5.4245035e-04 -1.7111382e-03 -2.3526847e-03 -1.6777874e-03
  6.8572012e-04 -4.4237674e-04 -1.7048402e-03  4.0355683e-03
  9.0814068e-04  1.7200888e-03  2.1919340e-04 -5.3816698e-03
  4.3119132e-03  5.2566440e-03 -3.7381786e-03 -8.2949642e-03
  2.4706360e-03 -7.3506031e-03  9.7903481e-05 -4.9497345e-03
 -5.6728041e-03  2.5019650e-03  2.9974235e-03  3.9790892e-03
  1.0027379e-03 -5.0721196e-03 -4.6809837e-03  6.3730031e-03
  6.9960661e-05  1.1619832e-03 -2.2614205e-03  4.7469474e-03
 -2.5865948e-03  1.5001415e-03 -2.1517400e-03  4.3217437e-03
 -1.1783938e-03  4.2903447e-03  7.2906259e-05  1.6983986e-03
 -5.2233655e-03  6.1700714e-04 -1.0928609e-03 -2.2263136e-03
 -6.0043153e-03  1.9090769e-03 -7.6657243e-04  1.2645293e-03
 -4.4988561e-03 -5.7830140e-03 -3.0025817e-03  3.1540622e-03
 -3.4476891e-03  6.8194291e-04  4.5258990e-03  1.3626209e-03
  6.561829

In [29]:
# If we want to determine the predicted character we need to sample the output distribution (pick a value based on probabillity)
sampled_indices = tf.random.categorical(pred, num_samples=1)

# now we can reshape that array and convert all the integers to numbers to see the actual characters
sampled_indices = np.reshape(sampled_indices, (1, -1))[0]
predicted_chars = int_to_text(sampled_indices)

predicted_chars  # and this is what the model predicted for training sequence 1

"Th $ZNS,xR;RT QYubUkXLZM,EI'.AawA,'G!-'yOB:r3G.nrFumsd$oPP&TYVIegmmCV?Lx vZ$qm BRBodiwW:OtATTBE?!.WJ"

In [21]:
def loss(labels, logits):
  return tf.keras.losses.sparse_categorical_crossentropy(labels, logits, from_logits=True)

In [22]:
model.compile(optimizer='adam', loss=loss)

In [23]:
# Directory where the checkpoints will be saved
checkpoint_dir = './training_checkpoints'
# Name of the checkpoint files
checkpoint_prefix = os.path.join(checkpoint_dir, "ckpt_{epoch}")

checkpoint_callback=tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_prefix,
    save_weights_only=True)

In [25]:
history = model.fit(data, epochs=20, callbacks=[checkpoint_callback])

Epoch 1/20
172/172 [==============================] - 12s 69ms/step - loss: 1.2691
Epoch 2/20
172/172 [==============================] - 12s 69ms/step - loss: 1.2355
Epoch 3/20
172/172 [==============================] - 12s 70ms/step - loss: 1.2030
Epoch 4/20
172/172 [==============================] - 12s 70ms/step - loss: 1.1692
Epoch 5/20
172/172 [==============================] - 12s 70ms/step - loss: 1.1344
Epoch 6/20
172/172 [==============================] - 12s 70ms/step - loss: 1.0993
Epoch 7/20
172/172 [==============================] - 12s 71ms/step - loss: 1.0618
Epoch 8/20
172/172 [==============================] - 12s 71ms/step - loss: 1.0231
Epoch 9/20
172/172 [==============================] - 12s 72ms/step - loss: 0.9831
Epoch 10/20
172/172 [==============================] - 12s 72ms/step - loss: 0.9435
Epoch 11/20
172/172 [==============================] - 12s 72ms/step - loss: 0.9044
Epoch 12/20
172/172 [==============================] - 12s 72ms/step - loss: 0.8647
E

In [26]:
model = build_model(VOCAB_SIZE, EMBEDDING_DIM, RNN_UNITS, batch_size=1)

In [27]:
model.load_weights(tf.train.latest_checkpoint(checkpoint_dir))
model.build(tf.TensorShape([1, None]))

In [ ]:
checkpoint_num = 10
model.load_weights(tf.train.load_checkpoint("./training_checkpoints/ckpt_" + str(checkpoint_num)))
model.build(tf.TensorShape([1, None]))

In [31]:
def generate_text(model, start_string):
  # Evaluation step (generating text using the learned model)

  # Number of characters to generate
  num_generate = 800

  # Converting our start string to numbers (vectorizing)
  input_eval = [char2idx[s] for s in start_string]
  input_eval = tf.expand_dims(input_eval, 0)

  # Empty string to store our results
  text_generated = []

  # Low temperatures results in more predictable text.
  # Higher temperatures results in more surprising text.
  # Experiment to find the best setting.
  temperature = 1.0

  # Here batch size == 1
  model.reset_states()
  for i in range(num_generate):
      predictions = model(input_eval)
      # remove the batch dimension
    
      predictions = tf.squeeze(predictions, 0)

      # using a categorical distribution to predict the character returned by the model
      predictions = predictions / temperature
      predicted_id = tf.random.categorical(predictions, num_samples=1)[-1,0].numpy()

      # We pass the predicted character as the next input to the model
      # along with the previous hidden state
      input_eval = tf.expand_dims([predicted_id], 0)

      text_generated.append(idx2char[predicted_id])

  return (start_string + ''.join(text_generated))

In [33]:
inp = input("Type a starting string: ")
print(generate_text(model, inp))

Type a starting string: king
kings, sits and fight.

Servant:
He were my man or foesand counsel: that
it confess I with some fever of grival bosom:
The provost knows your garlands; for thou shalt be
poor; but that we never meet thye any thing to me;
My insignance say at, thou hast an earth
His time let you not hore.

PROSPERO:
Not so leave, though beauty shall be to London.

JULIET:
Go with me, if he slew thy miserable.
Pruck to my counsel: think on the Tower,
Where you will part of any remember towns?
The respects for merry, grantame I do wear the crown. Romeo!
O, she had done--bouble youth,
Give me your worship for my price.
You twelve says
My fair Bianca, being to the sea
And credul dis in my part, for your grace, you would
Have danced by your desperate; young queen,
Which now for one more power.

CORIOLANUS:
I would r
